In [1]:
pip install dateparser

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pdfminer.six

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install langdetect

Note: you may need to restart the kernel to use updated packages.


# Filmijob Capstone Project: Interview Question Scoring

## Enriching the skills list with "Similar Skills" dataset to feed the parser with more information

In [4]:
import pandas as pd
df_skills= pd.read_csv("Skills_dataset/bigdataset_skills.csv")
df_skills.head()

,name,related_1,related_2,related_3,related_4,related_5,related_6,related_7,related_8,related_9,related_10
0,a certified,network certified,laptops,n certified,computer hardware,windows 7,comptia,troubleshooting,software installation,printers,xp
1,a certified it technician,n certified,network certified,comptia network certified,comptia a certification,a certified professional,mcdst,windows 7,computer hardware,computer repair,a certified
2,a certified professional,network certified,comptia network certified,n certified,microsoft certified professional,a certified it technician,troubleshooting,windows 7,active directory,computer hardware,windows server
3,a quick learner,product validation,great communication skills,vista,good communication skill,people person,project resourcing,new product release,touch typing,excellent communicator,internet savvy
4,a1 assessor,ptlls,assessor training,nvqs,training delivery,training needs analysis,trainers,train the trainer,employer engagement,commercial awareness,fleet


In [5]:
df_skills.dropna()

,name,related_1,related_2,related_3,related_4,related_5,related_6,related_7,related_8,related_9,related_10
0,a certified,network certified,laptops,n certified,computer hardware,windows 7,comptia,troubleshooting,software installation,printers,xp
1,a certified it technician,n certified,network certified,comptia network certified,comptia a certification,a certified professional,mcdst,windows 7,computer hardware,computer repair,a certified
2,a certified professional,network certified,comptia network certified,n certified,microsoft certified professional,a certified it technician,troubleshooting,windows 7,active directory,computer hardware,windows server
3,a quick learner,product validation,great communication skills,vista,good communication skill,people person,project resourcing,new product release,touch typing,excellent communicator,internet savvy
4,a1 assessor,ptlls,assessor training,nvqs,training delivery,training needs analysis,trainers,train the trainer,employer engagement,commercial awareness,fleet
...,...,...,...,...,...,...,...,...,...,...,...
28929,zsh,bash,git,linux,vim,python,ruby,emacs lisp,arch linux,javascript,postgresql
28930,zumba,aerobics,zumba instruction,certified group exercise instructor,pilates,salsa,fitness,fitness instruction,dance,latin dance,toning
28931,zumba instruction,zumba,dance instruction,fitness instruction,pilates instruction,latin dance,aerobics,group exercise instruction,cardio kickboxing,jazz dance,salsa
28932,zurb foundation,jquery,css,javascript,twitter bootstrap,sass,bootstrap,php,bower,css3,html 5


In [6]:
lista1 = df_skills['name'].tolist()
lista2 = df_skills['related_1'].tolist()
lista3 = df_skills['related_2'].tolist()
lista4 = df_skills['related_3'].tolist()
lista5 = df_skills['related_4'].tolist()
lista6 = df_skills['related_5'].tolist()
lista7 = df_skills['related_6'].tolist()
lista8 = df_skills['related_7'].tolist()
lista9 = df_skills['related_8'].tolist()
lista10 = df_skills['related_9'].tolist()
lista11 = df_skills['related_10'].tolist()

In [7]:
lista_total = lista1 + lista2 + lista3 + lista4 + lista5 + lista6 + lista7 + lista8 + lista9 + lista10 + lista11

In [8]:
myset = set(lista_total)

In [9]:
SKILLS_DB = ["teaching", "pharmaceutical", "erp", "fulfillment", "positioning", "tax", "service delivery", "investigate", "editorial", "account management", "business process", "valid drivers license", "electronics", "pr", "public relations", "javascript", "assembly", "digital marketing", "linux", "facebook", "spreadsheets", "recruit", "proposal", "sharepoint", "data entry", "hotel", "ordering", "branding", "life cycle", "real estate", "relationship management", "researching", "process improvements", "chemistry", "saas", "cad", "sales experience", "mathematics", "customer-facing", "audio", "project management skills", "six sigma", "hospitality", "mechanical engineering", "auditing", "employee relations", "android", "security clearance", "licensing", "adobe", "fundraising", "repairs", "iso", "market research", "warehouse", "business strategy", "pmp", "data management", "quality control", "reconciliation", "css", "conversion", "business analysis", "financial analysis", "ecommerce", "business intelligence", "c++", "client service", "publishing", "supervising", "complex projects", "key performance indicators", "scrum", "photoshop", "sports", "e-commerce", "journalism", "data collection", "higher education", "marketing programs", "financial management", "business plans", "user experience", "client relationships", "cloud", "analytical skills", "cisco", "internal stakeholders", "product marketing", "regulatory requirements", "itil", "information security", "aviation", "supply chain management", "python", "accounts payable", "industry experience", "autocad", "purchase orders", "acquisitions", "tv", "instrumentation", "strategic direction", "law enforcement", "call center", "experiments", "technical skills", "human resource", "business cases", "build relationships", "invoicing", "support services", "marketing strategy", "operating systems", "biology", "start-up", "electrical engineering", "workflows", "routing", "non-profit", "marketing plans", "due diligence", "business management", "iphone", "algorithms", "architectures", "reconcile", "dynamic environment", "external partners", "asset management", "microsoft word", "emea", "intranet", "sops", "sas", "digital media", "prospecting", "financial reporting", "project delivery", "seo", "operational excellence", "standard operating procedures", "c#", "technical knowledge", "on-call", "talent management", "stakeholder management", "tablets", "cms", "analyze data", "financial statements", "microsoft office suite", "fitness", "case management", "value proposition", "industry trends", "rfp", "broadcast", "portfolio management", "fabrication", "ux", "financial performance", "customer requirements", "psychology", "marketing materials", "resource management", "physics", "mortgage", "development activities", "end user", "business planning", "root cause", "analysis", "leadership development", "relationship building", "sdlc", "on-boarding", "quality standards", "regulatory compliance", "aws", "kpi", "status reports", "product line", "drafting", "jira", "phone calls", "product knowledge", "business stakeholders", "technical issues", "admissions", "supervisory experience", "usability", "pharmacy", "commissioning", "project plan", "ms excel", "fda", "test plans", "variances", "ui", "financing", "travel arrangements", "internal customers", "medical device", "counsel", "inventory management", "performance metrics", "lighting", "outsourcing", "indesign", "performance improvement", "management consulting", "graphic design", "transport", "information management", ".net", "startup", "matrix", "front-end", "project planning", "business systems", "accounts receivable", "public health", "hris", "german", "instructional design", "in-store", "data center", "matlab", "employee engagement", "cost effective", "sales management", "api", "adobe creative suite", "twitter", "program development", "event planning", "cash flow", "strategic plans", "vendor management", "trade shows", "hotels", "segmentation", "contract management", "gis", "talent acquisition", "photography", "internal communications", "client services", "ibm", "financial reports", "product quality", "beverage", "strong analytical skills", "underwriting", "cpr", "mining", "sales goals", "chemicals", "deposits", "scripting", "migration", "software engineering", "mis", "therapeutic", "general ledger", "tableau", "ms project", "standardization", "retention", "spelling", "media relations", "os", "daily operations", "immigration", "product design", "etl", "field sales", "driving record", "peoplesoft", "benchmark", "quality management", "apis", "test cases", "internal controls", "telecom", "business issues", "research projects", "data quality", "strategic initiatives", "office software", "cfa", "co-op", "big data", "journal entries", "vmware", "help desk", "statistical analysis", "datasets", "alliances", "solidworks", "prototype", "lan", "sci", "budget management", "unix", "rfps", "flex", "gaap", "experimental", "cpg", "information system", "customer facing", "process development", "web services", "international", "travel", "revenue growth", "software development life cycle", "operations management", "computer applications", "risk assessments", "sales operations", "raw materials", "internal audit", "physical security", "sql server", "affiliate", "computer software", "manage projects", "business continuity", "litigation", "it infrastructure", "cost reduction", "small business", "annual budget", "ios", "html5", "real-time", "consulting experience", "circuits", "machine learning", "risk assessment", "dns", "cross-functional team", "public policy", "analyzing data", "consulting services", "cinema", "arts", "recursos humanos", "excel", "accounting", "acquisition", "administrative support", "advertising", "agile", "analytical", "analytics", "architecture", "audit", "automation", "banking", "bi", "billing", "brand", "budget", "budgeting", "business administration", "business development", "business requirements", "c (programming language)", "certification", "change management", "coaching", "coding", "compliance", "computer science", "construction", "consulting", "consumers", "content", "continuous improvement", "contracts", "controls", "correspondence", "counseling", "crm", "customer experience", "customer service", "d (programming language)", "data analysis", "database", "design", "distribution", "documentation", "documenting", "drawings", "economics", "editing", "electrical", "engagement", "engineering", "expenses", "filing", "finance", "financial services", "forecasting", "forecasts", "frameworks", "governance", "hardware", "health", "healthcare", "hing", "hospital", "html", "i-deas", "information systems", "information technology", "installation", "internship", "inventory", "investigation", "investigations", "invoices", "java", "kpis", "lean", "legal", "legislation", "lifecycle", "logistics", "man resources", "management experience", "marketing", "merchandising", "metrics", "microsoft office", "mobile", "modeling", "ms office", "negotiation", "networking", "nursing", "operations", "oracle", "outreach", "partnership", "partnerships", "payments", "payroll", "performance management", "policies", "presentation", "presentations", "procurement", "product development", "product management", "program management", "programming", "project management", "protocols", "publications", "purchasing", "qa", "quality assurance", "r (programming language)", "recruiting", "recruitment", "regulations", "regulatory", "reporting", "research", "retail", "risk management", "safety", "sales", "salesforce", "sap", "scheduling", "social media", "software development", "sourcing", "specifications", "sql", "staffing", "statistics", "strategic planning", "strategy", "supply chain", "technical", "technical support", "testing", "training", "transactions", "transportation", "troubleshooting", "vendors", "video", "windows", "writing", "accomplished", "achieved", "acquired", "acted", "adapted", "addressed", "adjusted", "administered", "advanced", "advertised", "advised", "advocated", "aided", "aligned", "allocated", "amplified", "amplify", "analyzed", "answered", "applied", "appointed", "appraised", "approved", "arbitrated", "architected", "arranged", "articulated", "assembled", "assessed", "assigned", "assisted", "attained", "audited", "augmented", "authored", "authorized", "automated", "awarded", "balanced", "began", "blocked", "boosted", "brainstormed", "briefed", "budgeted", "built", "calculated", "campaigned", "capitalized", "captured", "cared", "cataloged", "categorized", "centralized", "chaired", "championed", "charted", "checked", "clarified", "classified", "co-authored", "coached", "coded", "collaborated", "collected", "combined", "communicated", "compared", "compiled", "completed", "composed", "computed", "conceptualized", "condensed", "conducted", "conferred", "conserved", "considered", "consolidated", "constructed", "consulted", "contacted", "contracted", "contributed", "controlled", "convert", "converted", "conveyed", "convinced", "cooperated", "coordinated", "corrected", "corresponded", "counseled", "crafted", "created", "creative", "critiqued", "cultivated", "curated", "customized", "debated", "debugged", "decided", "deciphered", "decreased", "deducted", "defined", "delegated", "delivered", "demonstrated", "deployed", "derived", "described", "designed", "detected", "determined", "developed", "devised", "diagnosed", "diagramed", "differentiated", "directed", "discerned", "discovered", "discussed", "dispatched", "dispensed", "displayed", "distributed", "documented", "drafted", "drew", "drove", "earned", "edited", "educated", "elicited", "eliminated", "emphasized", "empowered", "enabled", "enacted", "encouraged", "endeavored", "endorsed", "enforced", "engineered", "enhanced", "enlisted", "enlivened", "ensured", "entertained", "equalized", "established", "estimated", "evaluated", "examined", "exceeded", "executed", "expanded", "expedited", "experimented", "explained", "explored"\
           , "extracted", "fabricated", "facilitated", "familiarize", "fashioned", "fielded", "filed", "finalized", "fixed", "focused", "forecasted", "forged", "formalized", "formed", "formulated", "fortified", "fostered", "founded", "furnished", "furthered", "gained", "gathered", "generated", "goals", "grew", "guided", "halted", "handled", "headed", "helped", "helping", "hired", "hosted", "identified", "ignited", "illustrated", "imagined", "implemented", "improved", "incorporated", "increased", "individualized", "influenced", "informed", "initiated", "inspected", "inspired", "installed", "instilled", "instituted", "instructed", "insured", "integrated", "intensified", "interacted", "interpreted", "intervened", "interviewed", "introduced", "invented", "inventoried", "investigated", "involved", "itemized", "joined", "judged", "launched", "lectured", "led", "lessened", "leveraged", "lifted", "listened", "lobbied", "located", "logged", "lowered", "maintained", "managed", "mapped", "marketed", "maximized", "measured", "mediated", "mentored", "merged", "minimized", "mobilized", "modeled", "moderated", "modified", "monitored", "motivated", "navigated", "negotiated", "networked", "observed", "obtained", "operated", "optimized", "orchestrated", "ordered", "organized", "originated", "outlined", "outpaced", "outperformed", "overhauled", "oversaw", "participated", "partnered", "patched", "performed", "persuaded", "photographed", "piloted", "pioneered", "planned", "predicted", "prepared", "presented", "presided", "printed", "prioritized", "probed", "processed", "produced", "programmed", "projected", "promoted", "proofread", "proposed", "proved", "provided", "publicized", "published", "purchased", "qualified", "quantified", "raised", "reached", "rebuilt", "recognized", "recommended", "reconciled", "recorded", "recruited", "rectified", "redesigned", "reduced", "referred", "refined", "refocused", "registered", "regulated", "rehabilitated", "reinforced", "remodeled", "renovated", "reorganized", "repaired", "replaced", "reported", "researched", "reserved", "resolved", "responded", "restored", "restructured", "retrieved", "revamped", "reviewed", "revised", "revitalized", "rewrote", "routed", "saved", "scheduled", "screened", "scrutinized", "searched", "secured", "set", "shaped", "sharpened", "shattered", "showcased", "simplified", "simulated", "slashed", "solicited", "solved", "sparked", "spearheaded", "specialized", "specified", "spoke", "standardized", "steered", "stimulated", "storyboarded", "strategized", "streamlined", "strengthened", "studied", "submitted", "succeeded", "suggested", "summarized", "supervised", "supplied", "supported", "surpassed", "surveyed", "sustained", "synthesized", "systematized", "targeted", "taught", "terminated", "tested", "tracked", "trained", "transformed", "translated", "transmitted", "troubleshoot", "tutored", "unified", "united", "up", "updated", "upgraded", "utilized", "validated", "verified", "visualized", "volunteered", "won", "wrote", "yielded", "accountable", "accurate", "active", "adept", "alert", "amiable", "amicable", "ardent", "articulate", "assertive", "astute", "attentive", "bold", "businesslike", "calculating", "calm", "charming", "cheerful", "clear", "coherent", "cohesive", "collaborative", "committed", "competent", "complex", "concerted", "confident", "conscientious", "consistent", "constructive", "cooperative", "cost-effective", "courteous", "cutting-edge", "decisive", "dedicated", "dependable", "detail-oriented", "detailed", "devoted", "diligent", "diplomatic", "discerning", "disciplined", "driven", "earnest", "economical", "elegant", "eloquent", "energetic", "enterprising", "enthusiastic", "exact", "exhaustive", "expert", "fast-learning", "fastidious", "first-class", "fluent", "genuine", "groundbreaking", "harmonious", "honest", "ideal", "imaginative", "impartial", "industrious", "influential", "ingenious", "innovative", "insightful", "inspiring", "instrumental", "intelligent", "inventive", "investigative", "keen", "logical", "loyal", "methodical", "meticulous", "novel", "objective", "orderly", "painstaking", "passionate", "perceptive", "persistent", "personable", "persuasive", "pleasant", "positive", "practical", "precise", "proactive", "productive", "professional", "proficient", "profitable", "progressive", "punctual", "purposeful", "quality", "relentless", "reliable", "resilient", "respectful", "revolutionary", "robust", "rousing", "shrewd", "significant", "sincere", "skilled", "skillful", "smooth", "sophisticated", "spirited", "steadfast", "step-by-step", "strategic", "strong-willed", "structured", "studious", "superior", "supportive", "sustainable", "systematic", "tactful", "team-minded", "tenacious", "thoughtful", "timely", "tireless", "unique", "unprecedented", "vigorous", "visionary", "vivid", "well-grounded", "wholehearted", "world-class", "zealous"]

In [10]:
myset2 = set(SKILLS_DB)

In [11]:
def returnMatches(a,b):
       return list(set(a) & set(b))

In [12]:
total_set = SKILLS_DB + lista_total

In [13]:
Keywords = set(total_set)

In [14]:
mynewlist = list(Keywords)

In [15]:
mynewlist

[nan,
 'visual displays',
 'building business relationships',
 'rup',
 'race relations',
 'plc ladder logic',
 'addictive disorders',
 'bca protein assay',
 'medical practice operations',
 'oil on canvas',
 'osteopathy',
 'predictive modeling',
 'somatic experiencing',
 'audio system design',
 'exceptional public speaker',
 'travel arrangements',
 'business model',
 'hardware hacking',
 'quality',
 'project management office',
 'ees',
 'plant layout',
 'telephony',
 'medication errors',
 'sport marketing',
 'relaxation',
 'recalls',
 'front desk',
 'jack henry',
 'calcium imaging',
 'supervised',
 'plan de marketing',
 'workbench',
 'entertained',
 'sdn',
 'phpunit',
 'fmcsr',
 'loyalty',
 'return to work',
 'mortgage',
 'hair restoration',
 'hse',
 'music composition',
 'hrm',
 'erp implementation project management',
 'industria automobilistica',
 'digital imaging',
 'plaster casting',
 'asertividad',
 'operations research',
 'dfd',
 'protocolo bgp',
 'protein assays',
 'powder coati

In [16]:
df_list = pd.DataFrame (mynewlist, columns = ['keywords'])

In [17]:
df_list = df_list.dropna()

In [18]:
all_keywords = df_list['keywords'].to_list()

In [19]:
al = (map(lambda x: x.lower(), all_keywords))
all_keyword_lower= list(al)

In [20]:
all_keywords = all_keyword_lower

## FilmiJob CV Parser (Getting the keywords)

In [21]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO
import os 
from dateparser.search import search_dates #para identificar la fecha de nacimiento


import pandas as pd
import numpy as np
import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer

In [22]:
#INCLUDE HERE THE LINK TO THE CV OF THE CANDIDATE: Magda.pdf

In [23]:
def convert_pdf_to_txt(path, pages=None):
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)
    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    infile = open(path, 'rb')
    for page in PDFPage.get_pages(infile, pagenums):
        interpreter.process_page(page)
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close()
    return text

text = convert_pdf_to_txt('lmcv.pdf', pages=[0,1,2]) #Aquí añadiremos la ruta del CV

print(text)

LUIS MIGUEL VICTORIA VERAS 
Madrid, Spain | (+34) 672102432 | luismvictoriav@gmail.com | www.linkedin.com/in/luis-miguel-victoria-8896a71b7  

PROFESSIONAL SUMMARY 
Diligent with an in-depth knowledge of finance after working for 4 years in CCI Puesto de Bolsa, the biggest independent 
asset manager in the Dominican Republic. Strong technological skills enhanced and currently growing during the Masters 
in Big Data and Business Analytics. Significant critical thinking, logical reasoning and data visualization experience. Highly 
skilled in data analysis systems.  

EDUCATION 
IE SCHOOL OF HUMAN SCIENCES AND TECHNOLOGY  
Masters in Business Analytics and Big Data  
Relevant Courses 

•  PYTHON FOR DATA ANALYSIS 
•  SQL BASED DATA ARCHITECTURES 
•  MACHINE LEARNING 

SANTO DOMINGO INSTITUTE OF TECHNOLOGY  
Bachelor of Science, Commercial Engineering                     

PROFESSIONAL EXPERIENCE 
CCI PUESTO DE BOLSA (SECTOR, FINANCE  
Middle-Office Business Officer.  

           MADRID, 

In [24]:
# IDENTIFICAR IDIOMA DEL CV

from langdetect import detect
idiomacv = detect(text) #Identifica el idioma del CV entre 55 idiomas https://pypi.org/project/langdetect/
print("IDIOMA DEL CV: ",idiomacv)

IDIOMA DEL CV:  en


In [25]:
# IDENTIFICACIÓN DE HABILIDADES. Este dato también se puede usar para sacar keywords de las job descriptions

import nltk

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

nltk.download('stopwords')

# Importar aquí las KEYWORDS
SKILLS_DB = all_keywords

#["teaching", "pharmaceutical", "erp", "fulfillment", "positioning", "tax", "service delivery", "investigate", "editorial", "account management", "business process", "valid drivers license", "electronics", "pr", "public relations", "javascript", "assembly", "digital marketing", "linux", "facebook", "spreadsheets", "recruit", "proposal", "sharepoint", "data entry", "hotel", "ordering", "branding", "life cycle", "real estate", "relationship management", "researching", "process improvements", "chemistry", "saas", "cad", "sales experience", "mathematics", "customer-facing", "audio", "project management skills", "six sigma", "hospitality", "mechanical engineering", "auditing", "employee relations", "android", "security clearance", "licensing", "adobe", "fundraising", "repairs", "iso", "market research", "warehouse", "business strategy", "pmp", "data management", "quality control", "reconciliation", "css", "conversion", "business analysis", "financial analysis", "ecommerce", "business intelligence", "c++", "client service", "publishing", "supervising", "complex projects", "key performance indicators", "scrum", "photoshop", "sports", "e-commerce", "journalism", "data collection", "higher education", "marketing programs", "financial management", "business plans", "user experience", "client relationships", "cloud", "analytical skills", "cisco", "internal stakeholders", "product marketing", "regulatory requirements", "itil", "information security", "aviation", "supply chain management", "python", "accounts payable", "industry experience", "autocad", "purchase orders", "acquisitions", "tv", "instrumentation", "strategic direction", "law enforcement", "call center", "experiments", "technical skills", "human resource", "business cases", "build relationships", "invoicing", "support services", "marketing strategy", "operating systems", "biology", "start-up", "electrical engineering", "workflows", "routing", "non-profit", "marketing plans", "due diligence", "business management", "iphone", "algorithms", "architectures", "reconcile", "dynamic environment", "external partners", "asset management", "microsoft word", "emea", "intranet", "sops", "sas", "digital media", "prospecting", "financial reporting", "project delivery", "seo", "operational excellence", "standard operating procedures", "c#", "technical knowledge", "on-call", "talent management", "stakeholder management", "tablets", "cms", "analyze data", "financial statements", "microsoft office suite", "fitness", "case management", "value proposition", "industry trends", "rfp", "broadcast", "portfolio management", "fabrication", "ux", "financial performance", "customer requirements", "psychology", "marketing materials", "resource management", "physics", "mortgage", "development activities", "end user", "business planning", "root cause", "analysis", "leadership development", "relationship building", "sdlc", "on-boarding", "quality standards", "regulatory compliance", "aws", "kpi", "status reports", "product line", "drafting", "jira", "phone calls", "product knowledge", "business stakeholders", "technical issues", "admissions", "supervisory experience", "usability", "pharmacy", "commissioning", "project plan", "ms excel", "fda", "test plans", "variances", "ui", "financing", "travel arrangements", "internal customers", "medical device", "counsel", "inventory management", "performance metrics", "lighting", "outsourcing", "indesign", "performance improvement", "management consulting", "graphic design", "transport", "information management", ".net", "startup", "matrix", "front-end", "project planning", "business systems", "accounts receivable", "public health", "hris", "german", "instructional design", "in-store", "data center", "matlab", "employee engagement", "cost effective", "sales management", "api", "adobe creative suite", "twitter", "program development", "event planning", "cash flow", "strategic plans", "vendor management", "trade shows", "hotels", "segmentation", "contract management", "gis", "talent acquisition", "photography", "internal communications", "client services", "ibm", "financial reports", "product quality", "beverage", "strong analytical skills", "underwriting", "cpr", "mining", "sales goals", "chemicals", "deposits", "scripting", "migration", "software engineering", "mis", "therapeutic", "general ledger", "tableau", "ms project", "standardization", "retention", "spelling", "media relations", "os", "daily operations", "immigration", "product design", "etl", "field sales", "driving record", "peoplesoft", "benchmark", "quality management", "apis", "test cases", "internal controls", "telecom", "business issues", "research projects", "data quality", "strategic initiatives", "office software", "cfa", "co-op", "big data", "journal entries", "vmware", "help desk", "statistical analysis", "datasets", "alliances", "solidworks", "prototype", "lan", "sci", "budget management", "unix", "rfps", "flex", "gaap", "experimental", "cpg", "information system", "customer facing", "process development", "web services", "international", "travel", "revenue growth", "software development life cycle", "operations management", "computer applications", "risk assessments", "sales operations", "raw materials", "internal audit", "physical security", "sql server", "affiliate", "computer software", "manage projects", "business continuity", "litigation", "it infrastructure", "cost reduction", "small business", "annual budget", "ios", "html5", "real-time", "consulting experience", "circuits", "machine learning", "risk assessment", "dns", "cross-functional team", "public policy", "analyzing data", "consulting services", "cinema", "arts", "recursos humanos", "excel", "accounting", "acquisition", "administrative support", "advertising", "agile", "analytical", "analytics", "architecture", "audit", "automation", "banking", "bi", "billing", "brand", "budget", "budgeting", "business administration", "business development", "business requirements", "c (programming language)", "certification", "change management", "coaching", "coding", "compliance", "computer science", "construction", "consulting", "consumers", "content", "continuous improvement", "contracts", "controls", "correspondence", "counseling", "crm", "customer experience", "customer service", "d (programming language)", "data analysis", "database", "design", "distribution", "documentation", "documenting", "drawings", "economics", "editing", "electrical", "engagement", "engineering", "expenses", "filing", "finance", "financial services", "forecasting", "forecasts", "frameworks", "governance", "hardware", "health", "healthcare", "hing", "hospital", "html", "i-deas", "information systems", "information technology", "installation", "internship", "inventory", "investigation", "investigations", "invoices", "java", "kpis", "lean", "legal", "legislation", "lifecycle", "logistics", "man resources", "management experience", "marketing", "merchandising", "metrics", "microsoft office", "mobile", "modeling", "ms office", "negotiation", "networking", "nursing", "operations", "oracle", "outreach", "partnership", "partnerships", "payments", "payroll", "performance management", "policies", "presentation", "presentations", "procurement", "product development", "product management", "program management", "programming", "project management", "protocols", "publications", "purchasing", "qa", "quality assurance", "r (programming language)", "recruiting", "recruitment", "regulations", "regulatory", "reporting", "research", "retail", "risk management", "safety", "sales", "salesforce", "sap", "scheduling", "social media", "software development", "sourcing", "specifications", "sql", "staffing", "statistics", "strategic planning", "strategy", "supply chain", "technical", "technical support", "testing", "training", "transactions", "transportation", "troubleshooting", "vendors", "video", "windows", "writing", "accomplished", "achieved", "acquired", "acted", "adapted", "addressed", "adjusted", "administered", "advanced", "advertised", "advised", "advocated", "aided", "aligned", "allocated", "amplified", "amplify", "analyzed", "answered", "applied", "appointed", "appraised", "approved", "arbitrated", "architected", "arranged", "articulated", "assembled", "assessed", "assigned", "assisted", "attained", "audited", "augmented", "authored", "authorized", "automated", "awarded", "balanced", "began", "blocked", "boosted", "brainstormed", "briefed", "budgeted", "built", "calculated", "campaigned", "capitalized", "captured", "cared", "cataloged", "categorized", "centralized", "chaired", "championed", "charted", "checked", "clarified", "classified", "co-authored", "coached", "coded", "collaborated", "collected", "combined", "communicated", "compared", "compiled", "completed", "composed", "computed", "conceptualized", "condensed", "conducted", "conferred", "conserved", "considered", "consolidated", "constructed", "consulted", "contacted", "contracted", "contributed", "controlled", "convert", "converted", "conveyed", "convinced", "cooperated", "coordinated", "corrected", "corresponded", "counseled", "crafted", "created", "creative", "critiqued", "cultivated", "curated", "customized", "debated", "debugged", "decided", "deciphered", "decreased", "deducted", "defined", "delegated", "delivered", "demonstrated", "deployed", "derived", "described", "designed", "detected", "determined", "developed", "devised", "diagnosed", "diagramed", "differentiated", "directed", "discerned", "discovered", "discussed", "dispatched", "dispensed", "displayed", "distributed", "documented", "drafted", "drew", "drove", "earned", "edited", "educated", "elicited", "eliminated", "emphasized", "empowered", "enabled", "enacted", "encouraged", "endeavored", "endorsed", "enforced", "engineered", "enhanced", "enlisted", "enlivened", "ensured", "entertained", "equalized", "established", "estimated", "evaluated", "examined", "exceeded", "executed", "expanded", "expedited", "experimented", "explained", "explored"\
           #, "extracted", "fabricated", "facilitated", "familiarize", "fashioned", "fielded", "filed", "finalized", "fixed", "focused", "forecasted", "forged", "formalized", "formed", "formulated", "fortified", "fostered", "founded", "furnished", "furthered", "gained", "gathered", "generated", "goals", "grew", "guided", "halted", "handled", "headed", "helped", "helping", "hired", "hosted", "identified", "ignited", "illustrated", "imagined", "implemented", "improved", "incorporated", "increased", "individualized", "influenced", "informed", "initiated", "inspected", "inspired", "installed", "instilled", "instituted", "instructed", "insured", "integrated", "intensified", "interacted", "interpreted", "intervened", "interviewed", "introduced", "invented", "inventoried", "investigated", "involved", "itemized", "joined", "judged", "launched", "lectured", "led", "lessened", "leveraged", "lifted", "listened", "lobbied", "located", "logged", "lowered", "maintained", "managed", "mapped", "marketed", "maximized", "measured", "mediated", "mentored", "merged", "minimized", "mobilized", "modeled", "moderated", "modified", "monitored", "motivated", "navigated", "negotiated", "networked", "observed", "obtained", "operated", "optimized", "orchestrated", "ordered", "organized", "originated", "outlined", "outpaced", "outperformed", "overhauled", "oversaw", "participated", "partnered", "patched", "performed", "persuaded", "photographed", "piloted", "pioneered", "planned", "predicted", "prepared", "presented", "presided", "printed", "prioritized", "probed", "processed", "produced", "programmed", "projected", "promoted", "proofread", "proposed", "proved", "provided", "publicized", "published", "purchased", "qualified", "quantified", "raised", "reached", "rebuilt", "recognized", "recommended", "reconciled", "recorded", "recruited", "rectified", "redesigned", "reduced", "referred", "refined", "refocused", "registered", "regulated", "rehabilitated", "reinforced", "remodeled", "renovated", "reorganized", "repaired", "replaced", "reported", "researched", "reserved", "resolved", "responded", "restored", "restructured", "retrieved", "revamped", "reviewed", "revised", "revitalized", "rewrote", "routed", "saved", "scheduled", "screened", "scrutinized", "searched", "secured", "set", "shaped", "sharpened", "shattered", "showcased", "simplified", "simulated", "slashed", "solicited", "solved", "sparked", "spearheaded", "specialized", "specified", "spoke", "standardized", "steered", "stimulated", "storyboarded", "strategized", "streamlined", "strengthened", "studied", "submitted", "succeeded", "suggested", "summarized", "supervised", "supplied", "supported", "surpassed", "surveyed", "sustained", "synthesized", "systematized", "targeted", "taught", "terminated", "tested", "tracked", "trained", "transformed", "translated", "transmitted", "troubleshoot", "tutored", "unified", "united", "up", "updated", "upgraded", "utilized", "validated", "verified", "visualized", "volunteered", "won", "wrote", "yielded", "accountable", "accurate", "active", "adept", "alert", "amiable", "amicable", "ardent", "articulate", "assertive", "astute", "attentive", "bold", "businesslike", "calculating", "calm", "charming", "cheerful", "clear", "coherent", "cohesive", "collaborative", "committed", "competent", "complex", "concerted", "confident", "conscientious", "consistent", "constructive", "cooperative", "cost-effective", "courteous", "cutting-edge", "decisive", "dedicated", "dependable", "detail-oriented", "detailed", "devoted", "diligent", "diplomatic", "discerning", "disciplined", "driven", "earnest", "economical", "elegant", "eloquent", "energetic", "enterprising", "enthusiastic", "exact", "exhaustive", "expert", "fast-learning", "fastidious", "first-class", "fluent", "genuine", "groundbreaking", "harmonious", "honest", "ideal", "imaginative", "impartial", "industrious", "influential", "ingenious", "innovative", "insightful", "inspiring", "instrumental", "intelligent", "inventive", "investigative", "keen", "logical", "loyal", "methodical", "meticulous", "novel", "objective", "orderly", "painstaking", "passionate", "perceptive", "persistent", "personable", "persuasive", "pleasant", "positive", "practical", "precise", "proactive", "productive", "professional", "proficient", "profitable", "progressive", "punctual", "purposeful", "quality", "relentless", "reliable", "resilient", "respectful", "revolutionary", "robust", "rousing", "shrewd", "significant", "sincere", "skilled", "skillful", "smooth", "sophisticated", "spirited", "steadfast", "step-by-step", "strategic", "strong-willed", "structured", "studious", "superior", "supportive", "sustainable", "systematic", "tactful", "team-minded", "tenacious", "thoughtful", "timely", "tireless", "unique", "unprecedented", "vigorous", "visionary", "vivid", "well-grounded", "wholehearted", "world-class", "zealous"]

def extract_skills(input_text):
    if idiomacv == 'en':
        stop_words = set(nltk.corpus.stopwords.words('english')) # Habría que adaptarlo con los diferentes idiomas
    elif idiomacv == 'es':
        stop_words = set(nltk.corpus.stopwords.words('spanish'))
    word_tokens = nltk.tokenize.word_tokenize(input_text)

    # remove the stop words
    filtered_tokens = [w for w in word_tokens if w not in stop_words]

    # remove the punctuation
    filtered_tokens = [w for w in word_tokens if w.isalpha()]

    # generate bigrams and trigrams (such as artificial intelligence)
    bigrams_trigrams = list(map(' '.join, nltk.everygrams(filtered_tokens, 2, 3)))

    # we create a set to keep the results in.
    found_skills = set()

    # we search for each token in our skills database
    for token in filtered_tokens:
        if token.lower() in SKILLS_DB:
            found_skills.add(token)

    # we search for each bigram and trigram in our skills database
    for ngram in bigrams_trigrams:
        if ngram in SKILLS_DB:
            found_skills.add(ngram)

    return found_skills

if __name__ == '__main__':
    skills = extract_skills(text)

print("SKILLS: ", skills)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jdieg\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\jdieg\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\jdieg\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\jdieg\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jdieg\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


SKILLS:  {'Advanced', 'Analytics', 'Excel', 'reports', 'information', 'dashboard', 'ARCHITECTURES', 'marketing', 'Power', 'LANGUAGES', 'data visualization', 'Diligent', 'managed', 'trade', 'reasoning', 'PROFESSIONAL', 'Courses', 'ANALYSIS', 'CAPITAL', 'quality', 'Studio', 'agreements', 'client', 'documentation', 'EXPERIENCE', 'visualization', 'Commercial', 'performance', 'EDUCATION', 'Python', 'cash', 'logical', 'presentations', 'data analysis', 'vehicles', 'Science', 'Pandas', 'English', 'OTC', 'systems', 'SKILLS', 'Report', 'enhanced', 'forums', 'data', 'economic', 'Bi', 'PYTHON', 'manager', 'knowledge', 'performance metrics', 'HUMAN', 'DATA', 'Collected', 'Olympic', 'Data', 'promotional', 'finance', 'Processed', 'Prepared', 'quality control', 'French', 'independent', 'analysis', 'clients', 'company', 'experience', 'Engineering', 'asset', 'FINANCE', 'Updated', 'EXCHANGE', 'skills', 'Project', 'Fluent', 'critical thinking', 'participation', 'Basic', 'SQL', 'TECHNOLOGY', 'Spanish', 'Si

In [26]:
print(skills)
len(skills)

{'Advanced', 'Analytics', 'Excel', 'reports', 'information', 'dashboard', 'ARCHITECTURES', 'marketing', 'Power', 'LANGUAGES', 'data visualization', 'Diligent', 'managed', 'trade', 'reasoning', 'PROFESSIONAL', 'Courses', 'ANALYSIS', 'CAPITAL', 'quality', 'Studio', 'agreements', 'client', 'documentation', 'EXPERIENCE', 'visualization', 'Commercial', 'performance', 'EDUCATION', 'Python', 'cash', 'logical', 'presentations', 'data analysis', 'vehicles', 'Science', 'Pandas', 'English', 'OTC', 'systems', 'SKILLS', 'Report', 'enhanced', 'forums', 'data', 'economic', 'Bi', 'PYTHON', 'manager', 'knowledge', 'performance metrics', 'HUMAN', 'DATA', 'Collected', 'Olympic', 'Data', 'promotional', 'finance', 'Processed', 'Prepared', 'quality control', 'French', 'independent', 'analysis', 'clients', 'company', 'experience', 'Engineering', 'asset', 'FINANCE', 'Updated', 'EXCHANGE', 'skills', 'Project', 'Fluent', 'critical thinking', 'participation', 'Basic', 'SQL', 'TECHNOLOGY', 'Spanish', 'Significant

100

## EDA (Exploratory Data Analysis)

### Importing the roles dataset and checking the data

In [27]:
df=pd.read_csv("Dataset.csv")
df

,title,job description,questions
0,Collection Specialist,<h4>Job brief</h4><p>We are looking for a comp...,['A debtor has missed six months of payments. ...
1,Billing Analyst,<h4>Job brief</h4><p>We are looking for a Bill...,['How would you create a report on debts from ...
2,Tax Accountant,<h4>Job brief</h4><p>We are looking for a Tax ...,['What would you do if you received a B-Notice...
3,Accounts Receivable Manager,<h4>Job brief</h4><p>We are looking for an Acc...,['What would be the first thing you’d check if...
4,Cost Accountant,<h4>Job brief</h4><p>We are looking for a Cost...,"['If you could use only Excel, how would you o..."
...,...,...,...
278,Sales Advisor,"<h4>Job brief</h4><p><span style=""font-weight:...",['Imagine a customer asks you a question about...
279,Retail Buyer,"<h4>Job brief</h4><p><span style=""font-weight:...",['If I asked you to eliminate one line of our ...
280,Category Manager,"<h4>Job brief</h4><p><span style=""font-weight:...",['Here’s a spreadsheet with data. Can you spot...
281,Merchandiser,<h4>Job brief</h4><p>We are looking for an exp...,"['As a merchandiser, what do you think is the ..."


### Counting the number of keywords and checking for repeated keywords

In [28]:
#opened_file= open("keywords.txt")
from csv import reader
#read_file = reader(opened_file)
skills_data = list(skills)

In [29]:
skills_data

['Advanced',
 'Analytics',
 'Excel',
 'reports',
 'information',
 'dashboard',
 'ARCHITECTURES',
 'marketing',
 'Power',
 'LANGUAGES',
 'data visualization',
 'Diligent',
 'managed',
 'trade',
 'reasoning',
 'PROFESSIONAL',
 'Courses',
 'ANALYSIS',
 'CAPITAL',
 'quality',
 'Studio',
 'agreements',
 'client',
 'documentation',
 'EXPERIENCE',
 'visualization',
 'Commercial',
 'performance',
 'EDUCATION',
 'Python',
 'cash',
 'logical',
 'presentations',
 'data analysis',
 'vehicles',
 'Science',
 'Pandas',
 'English',
 'OTC',
 'systems',
 'SKILLS',
 'Report',
 'enhanced',
 'forums',
 'data',
 'economic',
 'Bi',
 'PYTHON',
 'manager',
 'knowledge',
 'performance metrics',
 'HUMAN',
 'DATA',
 'Collected',
 'Olympic',
 'Data',
 'promotional',
 'finance',
 'Processed',
 'Prepared',
 'quality control',
 'French',
 'independent',
 'analysis',
 'clients',
 'company',
 'experience',
 'Engineering',
 'asset',
 'FINANCE',
 'Updated',
 'EXCHANGE',
 'skills',
 'Project',
 'Fluent',
 'critical thinki

In [30]:
len(skills_data)

100

### Creating a new column with the number of question per role

First, we are going to do some cleaning on the questions in order to properly split them

In [31]:
ori60='[\'Using pseudo-code, implement binary search of a sorted array of integers\', \'What constitutes a good unit test and what a functional one?\', \'Describe the core principles of a REST API. How is this a different philosophy from RPC?\', \'Do arguments in Python get passed by reference or by value?\', \'Why are functions considered first class objects in Python?\', \'What tools do you use for linting, debugging and profiling?\', \'Give an example of filter and reduce over an iterable object\', \'What are list and dict comprehensions?\', \'What do we mean when we say that a certain Lambda expression forms a closure?\', \'Name a few differences between Python 2.x and 3.x\', \'How is memory managed in python?\', "What will be the output of the following code?\\nlist = [\'a\', \'b\', \'c\', \'d\', \'e\']print list[10:]list = [\'a\', \'b\', \'c\', \'d\', \'e\']\\nprint list[10:]list = [\'a\', \'b\', \'c\', \'d\', \'e\']\\nprint list[10:]\\n", \'Python uses a Global Interpreter Lock. Does that mean it doesn’t use real threads?\', \'Is it possible to have a producer thread reading from the network and a consumer thread writing to a file, really work in parallel? What about the GIL?\', \'A palindromic number reads the same both ways. The largest palindrome made from the product of two 2-digit numbers is 9009 = 91 × 99. Find the largest palindrome made from the product of two 3-digit numbers.\', \'What will be the output of the following code in each step?\\nclass C:  dangerous = 2c1 = C()c2 = C()print c1.dangerousc1.dangerous = 3print c1.dangerousprint c2.dangerousdel c1.dangerousprint c1.dangerousC.dangerous = 3print c2.dangerousclass C:\\n  dangerous = 2\\n\\nc1 = C()\\nc2 = C()\\nprint c1.dangerous\\n\\nc1.dangerous = 3\\nprint c1.dangerous\\nprint c2.dangerous\\n\\ndel c1.dangerous\\nprint c1.dangerous\\n\\nC.dangerous = 3\\nprint c2.dangerousclass C:\\n  dangerous = 2\\n\\nc1 = C()\\nc2 = C()\\nprint c1.dangerous\\n\\nc1.dangerous = 3\\nprint c1.dangerous\\nprint c2.dangerous\\n\\ndel c1.dangerous\\nprint c1.dangerous\\n\\nC.dangerous = 3\\nprint c2.dangerous\\n\']'
changed60='[\'Using pseudo-code, implement binary search of a sorted array of integers\', \'What constitutes a good unit test and what a functional one?\', \'Describe the core principles of a REST API. How is this a different philosophy from RPC?\', \'Do arguments in Python get passed by reference or by value?\', \'Why are functions considered first class objects in Python?\', \'What tools do you use for linting, debugging and profiling?\', \'Give an example of filter and reduce over an iterable object\', \'What are list and dict comprehensions?\', \'What do we mean when we say that a certain Lambda expression forms a closure?\', \'Name a few differences between Python 2.x and 3.x\', \'How is memory managed in python?\', "What will be the output of the following code?\\nlist = [\"a\", \"b\", \"c\", \"d\", \"e\"]print list[10:]list = [\"a\", \"b\", \"c\", \"d\", \"e\"]\\nprint list[10:]list = [\"a\", \"b\", \"c\", \"d\", \"e\"]\\nprint list[10:]\\n", \'Python uses a Global Interpreter Lock. Does that mean it doesn’t use real threads?\', \'Is it possible to have a producer thread reading from the network and a consumer thread writing to a file, really work in parallel? What about the GIL?\', \'A palindromic number reads the same both ways. The largest palindrome made from the product of two 2-digit numbers is 9009 = 91 × 99. Find the largest palindrome made from the product of two 3-digit numbers.\', \'What will be the output of the following code in each step?\\nclass C:  dangerous = 2c1 = C()c2 = C()print c1.dangerousc1.dangerous = 3print c1.dangerousprint c2.dangerousdel c1.dangerousprint c1.dangerousC.dangerous = 3print c2.dangerousclass C:\\n  dangerous = 2\\n\\nc1 = C()\\nc2 = C()\\nprint c1.dangerous\\n\\nc1.dangerous = 3\\nprint c1.dangerous\\nprint c2.dangerous\\n\\ndel c1.dangerous\\nprint c1.dangerous\\n\\nC.dangerous = 3\\nprint c2.dangerousclass C:\\n  dangerous = 2\\n\\nc1 = C()\\nc2 = C()\\nprint c1.dangerous\\n\\nc1.dangerous = 3\\nprint c1.dangerous\\nprint c2.dangerous\\n\\ndel c1.dangerous\\nprint c1.dangerous\\n\\nC.dangerous = 3\\nprint c2.dangerous\\n\']'
ori61='[\'Using pseudo-code, reverse a String iteratively and recursively\', \'What constitutes a good unit test and what a functional one?\', \'Do arguments in Python get passed by reference or by value?\', \'Why are functions considered first class objects in Python?\', \'What tools do you use for linting, debugging and profiling?\', \'Give an example of filter and reduce over an iterable object\', \'Implement the linux whereis command that locates the binary, source, and manual page files for a command.\', \'What are list and dict comprehensions?\', \'What do we mean when we say that a certain Lambda expression forms a closure?\', \'What is the difference between list and tuple?\', "What will be the output of the following code?\\nlist = [\'a\', \'b\', \'c\', \'d\', \'e\']print list[10:]list = [\'a\', \'b\', \'c\', \'d\', \'e\']\\nprint list[10:]list = [\'a\', \'b\', \'c\', \'d\', \'e\']\\nprint list[10:]\\n", \'What will be the output of the following code in each step?\\nclass C:  dangerous = 2c1 = C()c2 = C()print c1.dangerous c1.dangerous = 3print c1.dangerous print c2.dangerous del c1.dangerousprint c1.dangerous C.dangerous = 3print c2.dangerousclass C:\\n  dangerous = 2\\n\\nc1 = C()\\nc2 = C()\\nprint c1.dangerous \\n\\nc1.dangerous = 3\\nprint c1.dangerous \\nprint c2.dangerous \\n\\ndel c1.dangerous\\nprint c1.dangerous \\n\\nC.dangerous = 3\\nprint c2.dangerousclass C:\\n  dangerous = 2\\n\\nc1 = C()\\nc2 = C()\\nprint c1.dangerous \\n\\nc1.dangerous = 3\\nprint c1.dangerous \\nprint c2.dangerous \\n\\ndel c1.dangerous\\nprint c1.dangerous \\n\\nC.dangerous = 3\\nprint c2.dangerous\\n\\n\']'
changed61='[\'Using pseudo-code, reverse a String iteratively and recursively\', \'What constitutes a good unit test and what a functional one?\', \'Do arguments in Python get passed by reference or by value?\', \'Why are functions considered first class objects in Python?\', \'What tools do you use for linting, debugging and profiling?\', \'Give an example of filter and reduce over an iterable object\', \'Implement the linux whereis command that locates the binary, source, and manual page files for a command.\', \'What are list and dict comprehensions?\', \'What do we mean when we say that a certain Lambda expression forms a closure?\', \'What is the difference between list and tuple?\', "What will be the output of the following code?\\nlist = [\"a\", \"b\", \"c\", \"d\", \"e\"]print list[10:]list = [\"a\", \"b\", \"c\", \"d\", \"e\"]\\nprint list[10:]list = [\"a\", \"b\", \"c\", \"d\", \"e\"]\\nprint list[10:]\\n", \'What will be the output of the following code in each step?\\nclass C:  dangerous = 2c1 = C()c2 = C()print c1.dangerous c1.dangerous = 3print c1.dangerous print c2.dangerous del c1.dangerousprint c1.dangerous C.dangerous = 3print c2.dangerousclass C:\\n  dangerous = 2\\n\\nc1 = C()\\nc2 = C()\\nprint c1.dangerous \\n\\nc1.dangerous = 3\\nprint c1.dangerous \\nprint c2.dangerous \\n\\ndel c1.dangerous\\nprint c1.dangerous \\n\\nC.dangerous = 3\\nprint c2.dangerousclass C:\\n  dangerous = 2\\n\\nc1 = C()\\nc2 = C()\\nprint c1.dangerous \\n\\nc1.dangerous = 3\\nprint c1.dangerous \\nprint c2.dangerous \\n\\ndel c1.dangerous\\nprint c1.dangerous \\n\\nC.dangerous = 3\\nprint c2.dangerous\\n\\n\']'

In [32]:
df["questions"]=df["questions"].replace(ori60, changed60)
df["questions"]=df["questions"].replace(ori61, changed61)
df["questions"][61]

'[\'Using pseudo-code, reverse a String iteratively and recursively\', \'What constitutes a good unit test and what a functional one?\', \'Do arguments in Python get passed by reference or by value?\', \'Why are functions considered first class objects in Python?\', \'What tools do you use for linting, debugging and profiling?\', \'Give an example of filter and reduce over an iterable object\', \'Implement the linux whereis command that locates the binary, source, and manual page files for a command.\', \'What are list and dict comprehensions?\', \'What do we mean when we say that a certain Lambda expression forms a closure?\', \'What is the difference between list and tuple?\', "What will be the output of the following code?\\nlist = ["a", "b", "c", "d", "e"]print list[10:]list = ["a", "b", "c", "d", "e"]\\nprint list[10:]list = ["a", "b", "c", "d", "e"]\\nprint list[10:]\\n", \'What will be the output of the following code in each step?\\nclass C:  dangerous = 2c1 = C()c2 = C()print 

In [33]:
df["number_of_questions"]=df["questions"].str.split("', ?'").str.len()
df

,title,job description,questions,number_of_questions
0,Collection Specialist,<h4>Job brief</h4><p>We are looking for a comp...,['A debtor has missed six months of payments. ...,16
1,Billing Analyst,<h4>Job brief</h4><p>We are looking for a Bill...,['How would you create a report on debts from ...,14
2,Tax Accountant,<h4>Job brief</h4><p>We are looking for a Tax ...,['What would you do if you received a B-Notice...,14
3,Accounts Receivable Manager,<h4>Job brief</h4><p>We are looking for an Acc...,['What would be the first thing you’d check if...,17
4,Cost Accountant,<h4>Job brief</h4><p>We are looking for a Cost...,"['If you could use only Excel, how would you o...",15
...,...,...,...,...
278,Sales Advisor,"<h4>Job brief</h4><p><span style=""font-weight:...",['Imagine a customer asks you a question about...,16
279,Retail Buyer,"<h4>Job brief</h4><p><span style=""font-weight:...",['If I asked you to eliminate one line of our ...,19
280,Category Manager,"<h4>Job brief</h4><p><span style=""font-weight:...",['Here’s a spreadsheet with data. Can you spot...,15
281,Merchandiser,<h4>Job brief</h4><p>We are looking for an exp...,"['As a merchandiser, what do you think is the ...",15


In [34]:
df.describe()

,number_of_questions
count,283.000000
mean,15.452297
std,3.180228
min,8.000000
25%,13.000000
50%,15.000000
75%,17.000000
max,31.000000


On **average**, we have **15** questions per role. Also, the role with **less** questions has **7** and the role with the max questions has **29** questions.

### Cheking if there are any duplicated roles

In [35]:
all_jobs=df["title"].tolist()
all_jobs_set=set(all_jobs)
all_jobs_list=list(all_jobs_set)
len(all_jobs_list)

283

All the rows have a different role and no role is duplicated

### Checking if any of the columns has null values

In [36]:
df.isnull().any()

title                  False
job description        False
questions              False
number_of_questions    False
dtype: bool

No null values on the dataset

### Inspecting the total number of questions and questions repetitions

Now, we are goint to split the questions in every row of the questions column into a list of lists:

In [37]:
df["questions"]=df["questions"].str.split("', ?'")  
df

,title,job description,questions,number_of_questions
0,Collection Specialist,<h4>Job brief</h4><p>We are looking for a comp...,[['A debtor has missed six months of payments....,16
1,Billing Analyst,<h4>Job brief</h4><p>We are looking for a Bill...,[['How would you create a report on debts from...,14
2,Tax Accountant,<h4>Job brief</h4><p>We are looking for a Tax ...,[['What would you do if you received a B-Notic...,14
3,Accounts Receivable Manager,<h4>Job brief</h4><p>We are looking for an Acc...,[['What would be the first thing you’d check i...,17
4,Cost Accountant,<h4>Job brief</h4><p>We are looking for a Cost...,"[['If you could use only Excel, how would you ...",15
...,...,...,...,...
278,Sales Advisor,"<h4>Job brief</h4><p><span style=""font-weight:...",[['Imagine a customer asks you a question abou...,16
279,Retail Buyer,"<h4>Job brief</h4><p><span style=""font-weight:...",[['If I asked you to eliminate one line of our...,19
280,Category Manager,"<h4>Job brief</h4><p><span style=""font-weight:...",[['Here’s a spreadsheet with data. Can you spo...,15
281,Merchandiser,<h4>Job brief</h4><p>We are looking for an exp...,"[['As a merchandiser, what do you think is the...",15


In [39]:
#df["questions"]=df["questions"].str.split("[^a-e]', ?'[^a-e]")  
#df

After having a list of questions for every role, we are going to create a list of all the questions in the dataset:

In [38]:
all_questions=[]
for questions in df["questions"]:
    for question in questions:
        all_questions.append(question)
all_questions

["['A debtor has missed six months of payments. When you call the individual, they bring up financial difficulties and get emotional. How do you handle this?",
 'What payback plan would you recommend for a long-term client whose payments are due to be collected in X months?',
 'If you called a client to update them on their payment status and they were aggressive or rude to you, how would you handle it?',
 'Can you think of a few common excuses that debtors make? How would you respond to each of them?',
 'What collection methods do you know? Which technique do you think would be more effective for our company/clients?',
 'How do you keep historical data for each account?',
 'How often do you follow up with delinquent account holders and what’s your approach when you get in touch?',
 'Mention three things a Collection Specialist should avoid saying when speaking to a debtor.',
 'How do you keep track of all payments?',
 'What software do you use to increase your productivity?',
 'What d

In [39]:
len(all_questions)

4373

There are **4373** total questions

In [40]:
set_allquestions=set(all_questions)
unique_questions=list(set_allquestions)
len(unique_questions)

4169

There are **4169** different questions

In [41]:
num_repeated_questions=len(all_questions)-len(unique_questions)
num_repeated_questions

204

There are **204** question repetitions

Now, we are going to create a frequency table of questions to double check total number and repetitions:

In [42]:
questions_freq={}
for questions in df["questions"]:
    for question in questions:
        if question in questions_freq.keys():
            questions_freq[question]+=1
        else:
            questions_freq[question]=1
            
questions_freq 

{"['A debtor has missed six months of payments. When you call the individual, they bring up financial difficulties and get emotional. How do you handle this?": 1,
 'What payback plan would you recommend for a long-term client whose payments are due to be collected in X months?': 1,
 'If you called a client to update them on their payment status and they were aggressive or rude to you, how would you handle it?': 1,
 'Can you think of a few common excuses that debtors make? How would you respond to each of them?': 1,
 'What collection methods do you know? Which technique do you think would be more effective for our company/clients?': 1,
 'How do you keep historical data for each account?': 1,
 'How often do you follow up with delinquent account holders and what’s your approach when you get in touch?': 1,
 'Mention three things a Collection Specialist should avoid saying when speaking to a debtor.': 1,
 'How do you keep track of all payments?': 1,
 'What software do you use to increase yo

In this step we order the dictionary of questions in descending order to check the "most repeated" questions:

In [43]:
{k: v for k, v in sorted(questions_freq.items(), key=lambda item: item[1],reverse=True)}

{'How would you rate your computer skills?': 5,
 'How do you ensure accuracy in routine tasks such as processing expenses and preparing reports?': 4,
 'What constitutes a good unit test and what a functional one?': 4,
 'What accounting software have you used?': 3,
 'How familiar are you with SFAS (Statement of Financial Accounting Standards)?': 3,
 'How fast can you type?': 3,
 'Describe your experience with calendar management.': 3,
 'What tools do you use for linting, debugging and profiling?': 3,
 'What do we mean when we say that a certain Lambda expression forms a closure?': 3,
 'What tools & practices would you consider necessary for a Continuous Delivery solution?': 3,
 'Do arguments in Java get passed by reference or by value?': 3,
 'How are Runtime exceptions different from Checked exceptions?': 3,
 'Servlets 3.0 introduced async support. Describe a use case for it.': 3,
 'How do you troubleshoot a crashing application?': 3,
 'What’s your current occupation/What are you curren

In [44]:
count=0
repetitions=0
uniques=0
rep=[]
for key,value in questions_freq.items():
    if value>1:
        count+=1
        repetitions+=value
        rep.append(key)
    else:
        uniques+=1
print("There are",count,"questions with at least 2 appearances")
print("There are",uniques,"unique questions")
print("There are a total of ",(uniques+repetitions),"questions appearances")

There are 172 questions with at least 2 appearances
There are 3997 unique questions
There are a total of  4373 questions appearances


#### Putting all the question in lowercase

In [45]:
lower_questions=[]
for role in df["questions"]:
    role_list=[]
    for question in role:
        question=question.lower()
        role_list.append(question)
    lower_questions.append(role_list)
    
df["questions"]=lower_questions
len(df["questions"][0])

16

### Getting rid of the HTML code in the Job Description to make processing easier

In [46]:
df["job description"]=df["job description"].str.replace("<li>","")
df["job description"]=df["job description"].str.replace("</li>",", ")
df["job description"]=df["job description"].str.replace("</h4><p>",": ")
df["job description"]=df["job description"].str.replace("</h4><ul>",": ")
df["job description"]=df["job description"].str.replace("</p>","")
df["job description"]=df["job description"].str.replace("<h4>","")
df["job description"]=df["job description"].str.replace("</h4>",": ")
df["job description"]=df["job description"].str.replace("</ul>","")
df["job description"]=df["job description"].str.replace("\n","")
df["job description"]=df["job description"].str.replace("<p>"," ")
df["job description"]=df["job description"].str.replace('<span style="font-weight: 400;">',"")
df["job description"]=df["job description"].str.replace('</span><b>',"")
df["job description"]=df["job description"].str.replace('</span>',"")
df["job description"]=df["job description"].str.replace('</b>',"")
df["job description"]=df["job description"].str.replace(';,',",")
df["job description"]=df["job description"].str.replace('<p dir="ltr">',"")
df["job description"]=df["job description"].str.replace('<marketing-manager-job-description" rel="noopener" target="_blank">',"")
df["job description"]=df["job description"].str.replace('<marketing-specialist-job-description" rel="noopener" target="_blank">',"")
df["job description"]=df["job description"].str.replace('<marketing-officer-job-description" rel="noopener" target="_blank">',"")
df["job description"]=df["job description"].str.replace('<copywriter-job-description" rel="noopener" target="_blank">',"")
df["job description"]=df["job description"].str.replace('<data-analyst-job-description" rel="noopener" target="_blank">',"")
df["job description"]=df["job description"].str.replace('<accountant-job-description" >Accountants and <junior-accountant-job-description" >',"")
df["job description"]=df["job description"].str.replace('<financial-controller-job-description" >',"")
df["job description"]=df["job description"].str.replace('<financial-manager-job-description" >',"")
df["job description"]=df["job description"].str.replace('<accounting-manager-job-description" >',"")
df["job description"]=df["job description"].str.replace('<accounting-supervisor-job-description">',"")
df["job description"]=df["job description"].str.replace('<management-accountant-job-description" >',"")
df["job description"]=df["job description"].str.replace('<certified-public-accountant-cpa-job-description">',"")
df["job description"]=df["job description"].str.replace('<accounting-clerk-job-description">',"")
df["job description"]=df["job description"].str.replace('<medical-secretary-job-description">',"")
df["job description"]=df["job description"].str.replace('<span class="hardreadability"><span data-offset-key="65n7a-0-0"><span data-text="true">',"")


In [47]:
df["job description"][198]

'Job brief: We are looking for an experienced Procurement Manager to manage the company’s supply of products and services<span data-offset-key="65n7a-1-0"><span data-text="true">. Procurement Manager responsibilities include strategizing to\xa0find cost-effective deals and suppliers.\xa0<span class="hardreadability"><span data-offset-key="65n7a-2-0"><span data-text="true">The Procurement Manager’s duty\xa0is to discover the best\xa0ways to cut procurement expenses, so that the company can invest in growth and people<span data-offset-key="65n7a-3-0"><span data-text="true">.Responsibilities: <span data-offset-key="65n7a-0-0"><span data-text="true">Devise and use fruitful <sourcing-manager-job-description"><span data-offset-key="65n7a-1-0"><span data-text="true">sourcing strategies, <li class="public-DraftStyleDefault-unorderedListItem public-DraftStyleDefault-depth0 public-DraftStyleDefault-listLTR" data-block="true" data-editor="98src" data-offset-key="1qema-0-0"><div class="public-Draf

## Feature Engineering

### Enriching the keywords list with word2vec embeddings

In [48]:
import gensim.models.keyedvectors as word2vec

In [49]:
path = "C:/Users/jdieg/OneDrive/Desktop/IE Master in Business Analytics and Big Data/Term 3/Capstone Project/Pre-Trained Word2Vec/GoogleNews-vectors-negative300.bin"
#path = "C:\Users\jdieg\OneDrive\Desktop\IE Master in Business Analytics and Big Data\Term 3\Capstone Project\Pre-Trained Word2Vec\GoogleNews-vectors-negative300.bin"

#loading the downloaded model
model = word2vec.KeyedVectors.load_word2vec_format(path, binary=True) 

In [50]:
print(skills_data)
len(skills_data)

['Advanced', 'Analytics', 'Excel', 'reports', 'information', 'dashboard', 'ARCHITECTURES', 'marketing', 'Power', 'LANGUAGES', 'data visualization', 'Diligent', 'managed', 'trade', 'reasoning', 'PROFESSIONAL', 'Courses', 'ANALYSIS', 'CAPITAL', 'quality', 'Studio', 'agreements', 'client', 'documentation', 'EXPERIENCE', 'visualization', 'Commercial', 'performance', 'EDUCATION', 'Python', 'cash', 'logical', 'presentations', 'data analysis', 'vehicles', 'Science', 'Pandas', 'English', 'OTC', 'systems', 'SKILLS', 'Report', 'enhanced', 'forums', 'data', 'economic', 'Bi', 'PYTHON', 'manager', 'knowledge', 'performance metrics', 'HUMAN', 'DATA', 'Collected', 'Olympic', 'Data', 'promotional', 'finance', 'Processed', 'Prepared', 'quality control', 'French', 'independent', 'analysis', 'clients', 'company', 'experience', 'Engineering', 'asset', 'FINANCE', 'Updated', 'EXCHANGE', 'skills', 'Project', 'Fluent', 'critical thinking', 'participation', 'Basic', 'SQL', 'TECHNOLOGY', 'Spanish', 'Significant

100

In [51]:
expanded_keywords=[]
for keyword in skills_data:
    expanded_keywords.append(keyword)
    try:
         if (" " not in keyword):
                expanded_words=model.most_similar(keyword)
                for word in expanded_words:
                    if word[1]>0.6:
                        expanded_keywords.append(word[0])
    except KeyError:
        pass
            
print(expanded_keywords)
print(len(expanded_keywords))

['Advanced', 'Systems_Concepts_ASCI', 'Analytics', 'analytics', 'Excel', 'Microsoft_Excel', 'MS_Excel', 'Excel_spreadsheet', 'Visio', 'Excel_spreadsheets', 'reports', 'Reports', 'repots', 'reported', 'report', 'information', 'info', 'infomation', 'infor_mation', 'informaiton', 'informa_tion', 'informationon', 'informationabout', 'Information', 'informaion', 'details', 'dashboard', 'dashboards', 'Dashboard', 'instrument_cluster', 'ARCHITECTURES', 'marketing', 'Marketing', 'branding', 'merchandising', 'Respond2', 'Power', 'LANGUAGES', 'COVERAGE_FOR', 'DO_YOU_NEED', 'THE_VACCINE', 'TRENDS_IN', 'WITH_OTHER', 'CONCERN_OVER', 'BENEFITS_FROM', 'PROTOCOLS', 'ARMENIANS', 'RELUCTANT_TO', 'data visualization', 'Diligent', 'managed', 'manged', 'trade', 'reasoning', 'logic', 'rationale', 'logical_reasoning', 'PROFESSIONAL', 'CONSULTANT', 'PROFESSIONALS', 'ASSISTANT', 'PRODUCTIVE', 'NEW_CAREER', 'TRAINING_AND', 'FOR_SUCCESS', 'Courses', 'courses', 'Training_Courses', 'Classes', 'Course', 'Seminars',

In [50]:
#expanded_keywords=[]
#for keyword in keywords_data:
    #expanded_keywords.append(keyword)
    #if (" " not in keyword) & (keyword in model.most_similar(keyword)):
        #expanded_words=model.most_similar(keyword)
        #for word in expanded_words:
            #if word[1]>0.6:
                #expanded_keywords.append(word[0])
            
#print(expanded_keywords)
#print(len(expanded_keywords))

### Normalization of keywords

#### Applying Lemmatization to the keywords list

In [52]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
lemmatized_keywords_list=[]
for keyword in expanded_keywords:
    lemmatized_keywords_list.append(lemmatizer.lemmatize(keyword))

In [53]:
print(lemmatized_keywords_list)
len(lemmatized_keywords_list)

['Advanced', 'Systems_Concepts_ASCI', 'Analytics', 'analytics', 'Excel', 'Microsoft_Excel', 'MS_Excel', 'Excel_spreadsheet', 'Visio', 'Excel_spreadsheets', 'report', 'Reports', 'repots', 'reported', 'report', 'information', 'info', 'infomation', 'infor_mation', 'informaiton', 'informa_tion', 'informationon', 'informationabout', 'Information', 'informaion', 'detail', 'dashboard', 'dashboard', 'Dashboard', 'instrument_cluster', 'ARCHITECTURES', 'marketing', 'Marketing', 'branding', 'merchandising', 'Respond2', 'Power', 'LANGUAGES', 'COVERAGE_FOR', 'DO_YOU_NEED', 'THE_VACCINE', 'TRENDS_IN', 'WITH_OTHER', 'CONCERN_OVER', 'BENEFITS_FROM', 'PROTOCOLS', 'ARMENIANS', 'RELUCTANT_TO', 'data visualization', 'Diligent', 'managed', 'manged', 'trade', 'reasoning', 'logic', 'rationale', 'logical_reasoning', 'PROFESSIONAL', 'CONSULTANT', 'PROFESSIONALS', 'ASSISTANT', 'PRODUCTIVE', 'NEW_CAREER', 'TRAINING_AND', 'FOR_SUCCESS', 'Courses', 'course', 'Training_Courses', 'Classes', 'Course', 'Seminars', 'AN

514

In [54]:
expanded_keywords.extend(lemmatized_keywords_list)
print(expanded_keywords)
print(len(expanded_keywords))

['Advanced', 'Systems_Concepts_ASCI', 'Analytics', 'analytics', 'Excel', 'Microsoft_Excel', 'MS_Excel', 'Excel_spreadsheet', 'Visio', 'Excel_spreadsheets', 'reports', 'Reports', 'repots', 'reported', 'report', 'information', 'info', 'infomation', 'infor_mation', 'informaiton', 'informa_tion', 'informationon', 'informationabout', 'Information', 'informaion', 'details', 'dashboard', 'dashboards', 'Dashboard', 'instrument_cluster', 'ARCHITECTURES', 'marketing', 'Marketing', 'branding', 'merchandising', 'Respond2', 'Power', 'LANGUAGES', 'COVERAGE_FOR', 'DO_YOU_NEED', 'THE_VACCINE', 'TRENDS_IN', 'WITH_OTHER', 'CONCERN_OVER', 'BENEFITS_FROM', 'PROTOCOLS', 'ARMENIANS', 'RELUCTANT_TO', 'data visualization', 'Diligent', 'managed', 'manged', 'trade', 'reasoning', 'logic', 'rationale', 'logical_reasoning', 'PROFESSIONAL', 'CONSULTANT', 'PROFESSIONALS', 'ASSISTANT', 'PRODUCTIVE', 'NEW_CAREER', 'TRAINING_AND', 'FOR_SUCCESS', 'Courses', 'courses', 'Training_Courses', 'Classes', 'Course', 'Seminars',

#### Putting all keywords in lowercase

In [55]:
a = (map(lambda x: x.lower(), expanded_keywords))
lower_keyword_list= list(a)
print(lower_keyword_list)
len(lower_keyword_list)

['advanced', 'systems_concepts_asci', 'analytics', 'analytics', 'excel', 'microsoft_excel', 'ms_excel', 'excel_spreadsheet', 'visio', 'excel_spreadsheets', 'reports', 'reports', 'repots', 'reported', 'report', 'information', 'info', 'infomation', 'infor_mation', 'informaiton', 'informa_tion', 'informationon', 'informationabout', 'information', 'informaion', 'details', 'dashboard', 'dashboards', 'dashboard', 'instrument_cluster', 'architectures', 'marketing', 'marketing', 'branding', 'merchandising', 'respond2', 'power', 'languages', 'coverage_for', 'do_you_need', 'the_vaccine', 'trends_in', 'with_other', 'concern_over', 'benefits_from', 'protocols', 'armenians', 'reluctant_to', 'data visualization', 'diligent', 'managed', 'manged', 'trade', 'reasoning', 'logic', 'rationale', 'logical_reasoning', 'professional', 'consultant', 'professionals', 'assistant', 'productive', 'new_career', 'training_and', 'for_success', 'courses', 'courses', 'training_courses', 'classes', 'course', 'seminars',

1028

In [57]:
keyword_set=set(lower_keyword_list)
unique_lower_keyword_list=list(keyword_set)
len(unique_lower_keyword_list)

462

### Creating a new column with the list of keywords

In [60]:
df['keywords'] = [unique_lower_keyword_list for i in df.index]
df.head()

,title,job description,questions,number_of_questions,keywords
0,Collection Specialist,Job brief: We are looking for a competent Coll...,[['a debtor has missed six months of payments....,16,"[coverage_for, instrument_cluster, comapny, da..."
1,Billing Analyst,Job brief: We are looking for a Billing Analys...,[['how would you create a report on debts from...,14,"[coverage_for, instrument_cluster, comapny, da..."
2,Tax Accountant,Job brief: We are looking for a Tax Accountant...,[['what would you do if you received a b-notic...,14,"[coverage_for, instrument_cluster, comapny, da..."
3,Accounts Receivable Manager,Job brief: We are looking for an Accounts Rece...,[['what would be the first thing you’d check i...,17,"[coverage_for, instrument_cluster, comapny, da..."
4,Cost Accountant,Job brief: We are looking for a Cost Accountan...,"[['if you could use only excel, how would you ...",15,"[coverage_for, instrument_cluster, comapny, da..."


### Grouping Keywords with job descriptions

**This step MAY NOT be applied**

#### Putting job descriptions in lower case

In [62]:
lower_job_description=[]
for role in df["job description"]:
    role=role.lower()
    lower_job_description.append(role)
print(lower_job_description[0])
df["job description"]=lower_job_description

job brief: we are looking for a competent collection specialist to contact clients and collect outstanding payments. you will struck a balance between maintaining trustful relationships, and ensuring timely payments. our collection specialist should exhibit professionalism and trustworthiness. you should have excellent communication and negotiation skills, as well as an ability to work independently.responsibilities: monitor accounts to identify outstanding debts, investigate historical data for each debt or bill, find and contact clients to ask about their overdue payments, take actions to encourage timely debt payments, process payments and refunds, resolve billing and customer credit issues, update account status records and collection efforts, report on collection activity and accounts receivable status, requirements: <collection-specialist-interview-questions">proven experience as a collection specialist or similar role, knowledge of billing procedures and collection techniques (e

In [63]:
df["keyword_with_jobdesc"] = df["job description"].str.findall('|'.join(lower_keyword_list)).apply(set).str.join(', ')
df.head(20)

,title,job description,questions,number_of_questions,keywords,keyword_with_jobdesc
0,Collection Specialist,job brief: we are looking for a competent coll...,[['a debtor has missed six months of payments....,16,"[coverage_for, instrument_cluster, comapny, da...","experience, bi, truck, ability, independent, s..."
1,Billing Analyst,job brief: we are looking for a billing analys...,[['how would you create a report on debts from...,14,"[coverage_for, instrument_cluster, comapny, da...","experience, bi, organizational, contracts, abi..."
2,Tax Accountant,job brief: we are looking for a tax accountant...,[['what would you do if you received a b-notic...,14,"[coverage_for, instrument_cluster, comapny, da...","experience, bi, detail, business, financial, r..."
3,Accounts Receivable Manager,job brief: we are looking for an accounts rece...,[['what would be the first thing you’d check i...,17,"[coverage_for, instrument_cluster, comapny, da...","experience, bi, advanced, ability, detail, cli..."
4,Cost Accountant,job brief: we are looking for a cost accountan...,"[['if you could use only excel, how would you ...",15,"[coverage_for, instrument_cluster, comapny, da...","bi, reports, information, detail, finance, rep..."
5,Certified Public Accountant (CPA),job brief: we are looking for an experienced c...,[['if you missed the deadline for paying our c...,15,"[coverage_for, instrument_cluster, comapny, da...","bi, reports, information, investment, particip..."
6,Accounting Supervisor,job brief: we are looking for an accounting su...,[['what measures would you take if you discove...,13,"[coverage_for, instrument_cluster, comapny, da...","bi, reports, performance, sheet, information, ..."
7,Senior Auditor,job brief: we are looking for a senior auditor...,[['what would you do if a client/manager asked...,14,"[coverage_for, instrument_cluster, comapny, da...","organizational, bi, experience, ability, detai..."
8,Billing Specialist,job brief: we are looking for a competent bill...,[['describe your typical day at your current/p...,14,"[coverage_for, instrument_cluster, comapny, da...","info, experience, bi, detail, client, financia..."
9,Billing Clerk,job brief: we are looking for a reliable billi...,[['what data would you present to inform your ...,16,"[coverage_for, instrument_cluster, comapny, da...","bi, information, cash, customer, school, educa..."


Once we have the keywords that match the job description, we create a list out of them for further processing:

In [64]:
df["keyword_with_jobdesc"]=df["keyword_with_jobdesc"].str.split(", ")
df.head(20)

,title,job description,questions,number_of_questions,keywords,keyword_with_jobdesc
0,Collection Specialist,job brief: we are looking for a competent coll...,[['a debtor has missed six months of payments....,16,"[coverage_for, instrument_cluster, comapny, da...","[experience, bi, truck, ability, independent, ..."
1,Billing Analyst,job brief: we are looking for a billing analys...,[['how would you create a report on debts from...,14,"[coverage_for, instrument_cluster, comapny, da...","[experience, bi, organizational, contracts, ab..."
2,Tax Accountant,job brief: we are looking for a tax accountant...,[['what would you do if you received a b-notic...,14,"[coverage_for, instrument_cluster, comapny, da...","[experience, bi, detail, business, financial, ..."
3,Accounts Receivable Manager,job brief: we are looking for an accounts rece...,[['what would be the first thing you’d check i...,17,"[coverage_for, instrument_cluster, comapny, da...","[experience, bi, advanced, ability, detail, cl..."
4,Cost Accountant,job brief: we are looking for a cost accountan...,"[['if you could use only excel, how would you ...",15,"[coverage_for, instrument_cluster, comapny, da...","[bi, reports, information, detail, finance, re..."
5,Certified Public Accountant (CPA),job brief: we are looking for an experienced c...,[['if you missed the deadline for paying our c...,15,"[coverage_for, instrument_cluster, comapny, da...","[bi, reports, information, investment, partici..."
6,Accounting Supervisor,job brief: we are looking for an accounting su...,[['what measures would you take if you discove...,13,"[coverage_for, instrument_cluster, comapny, da...","[bi, reports, performance, sheet, information,..."
7,Senior Auditor,job brief: we are looking for a senior auditor...,[['what would you do if a client/manager asked...,14,"[coverage_for, instrument_cluster, comapny, da...","[organizational, bi, experience, ability, deta..."
8,Billing Specialist,job brief: we are looking for a competent bill...,[['describe your typical day at your current/p...,14,"[coverage_for, instrument_cluster, comapny, da...","[info, experience, bi, detail, client, financi..."
9,Billing Clerk,job brief: we are looking for a reliable billi...,[['what data would you present to inform your ...,16,"[coverage_for, instrument_cluster, comapny, da...","[bi, information, cash, customer, school, educ..."


### Create a counter of keywords per question

In [69]:
column_list=[]
question_key_count={}
for role in df["questions"]:
    new_dict={}
    for question in role:
        new_dict[question]=0
    column_list.append(new_dict)

i=0
keyword_count=0
for role_dict in column_list:  
    for question in role_dict.keys():
        for keyword in df["keywords"][i]: 
            if keyword in question:  
                keyword_count+=1
        role_dict[question]=keyword_count
        keyword_count=0
    i+=1
            
print(column_list[1])
print(len(column_list))

{"['how would you create a report on debts from the previous quarter?": 2, 'how would you contact clients who haven’t paid their bills?': 3, 'if a long-term client had a debt at the end of the fiscal year, how would you address it with him/her?': 1, 'how would you make sure we collect invoices in a timely manner?': 0, 'how would you handle an invoice discrepancy with a client?': 1, 'briefly explain the financial transactions involved in the billing process and your experience with each.': 3, 'what accounting software have you used?': 0, 'how do you keep updated records of accounts receivable from clients?': 3, 'how do you ensure financial information remains confidential?': 3, 'what are your responsibilities during annual audits?': 1, 'have you ever made a mistake at work (e.g. a wrong calculation)? how did you figure it out and what happened?': 0, 'how do you manage to meet tight deadlines?': 0, 'how do you stay up-to-date with new financial regulations? do you read any specific blogs

In [70]:
df["keywords_per_question"]=column_list
df.head()

,title,job description,questions,number_of_questions,keywords,keyword_with_jobdesc,keywords_per_question
0,Collection Specialist,job brief: we are looking for a competent coll...,[['a debtor has missed six months of payments....,16,"[coverage_for, instrument_cluster, comapny, da...","[experience, bi, truck, ability, independent, ...",{'['a debtor has missed six months of payments...
1,Billing Analyst,job brief: we are looking for a billing analys...,[['how would you create a report on debts from...,14,"[coverage_for, instrument_cluster, comapny, da...","[experience, bi, organizational, contracts, ab...",{'['how would you create a report on debts fro...
2,Tax Accountant,job brief: we are looking for a tax accountant...,[['what would you do if you received a b-notic...,14,"[coverage_for, instrument_cluster, comapny, da...","[experience, bi, detail, business, financial, ...",{'['what would you do if you received a b-noti...
3,Accounts Receivable Manager,job brief: we are looking for an accounts rece...,[['what would be the first thing you’d check i...,17,"[coverage_for, instrument_cluster, comapny, da...","[experience, bi, advanced, ability, detail, cl...",{'['what would be the first thing you’d check ...
4,Cost Accountant,job brief: we are looking for a cost accountan...,"[['if you could use only excel, how would you ...",15,"[coverage_for, instrument_cluster, comapny, da...","[bi, reports, information, detail, finance, re...","{'['if you could use only excel, how would you..."


In [71]:
df["keywords_per_question"][1]

{"['how would you create a report on debts from the previous quarter?": 2,
 'how would you contact clients who haven’t paid their bills?': 3,
 'if a long-term client had a debt at the end of the fiscal year, how would you address it with him/her?': 1,
 'how would you make sure we collect invoices in a timely manner?': 0,
 'how would you handle an invoice discrepancy with a client?': 1,
 'briefly explain the financial transactions involved in the billing process and your experience with each.': 3,
 'what accounting software have you used?': 0,
 'how do you keep updated records of accounts receivable from clients?': 3,
 'how do you ensure financial information remains confidential?': 3,
 'what are your responsibilities during annual audits?': 1,
 'have you ever made a mistake at work (e.g. a wrong calculation)? how did you figure it out and what happened?': 0,
 'how do you manage to meet tight deadlines?': 0,
 'how do you stay up-to-date with new financial regulations? do you read any sp

### Create a TFIDF for the questions words in order to get a score of them

In [72]:
df_test=pd.read_csv("Dataset.csv")

In [73]:
tfidf_questions=df_test["questions"]

In [74]:
tf_idf_weighting = TfidfVectorizer()
tf_idf_test = tf_idf_weighting.fit_transform(tfidf_questions)
tf_idf_dt_matrix = pd.DataFrame(tf_idf_test.A, columns=tf_idf_weighting.get_feature_names())
tf_idf_dt_matrix

,00,000,10,100,1096,1099,12,14,14001,15,...,yes,yet,yii,you,young,younger,your,yourself,zoho,zone
0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.399921,0.0,0.0,0.050953,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.319325,0.0,0.0,0.048434,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.122246,0.122246,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.369459,0.0,0.0,0.108962,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.419516,0.0,0.0,0.121476,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.086231,0.000000,0.0,0.372815,0.0,0.0,0.018849,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.096141,0.0,0.529787,0.0,0.0,0.000000,0.0,0.0,0.0
279,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.327401,0.0,0.0,0.109772,0.0,0.0,0.0
280,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.249732,0.0,0.0,0.033143,0.0,0.0,0.0
281,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.269223,0.0,0.0,0.153914,0.0,0.0,0.0


In [75]:
document = 2
print("25 most important terms for role", df_test.iloc[document]['title'])
print(tf_idf_dt_matrix.iloc[:, np.argsort(tf_idf_dt_matrix.loc[document])[::-1]].iloc[document][:25])

print("25 least important terms for role", df_test.iloc[document]['title'])
print(tf_idf_dt_matrix.iloc[:, np.argsort(tf_idf_dt_matrix.loc[document])[::-1]].iloc[document][-25:])

25 most important terms for role Tax Accountant
you            0.369459
tax            0.281376
notice         0.206878
regulatory     0.193066
what           0.164204
accounting     0.146057
do             0.124023
create         0.123621
how            0.123588
1099           0.122246
1096           0.122246
withholding    0.122246
actually       0.122246
submission     0.122246
attracted      0.122246
forensic       0.122246
reports        0.120501
versus         0.113924
fields         0.113924
irs            0.113924
aware          0.113924
backup         0.113924
returns        0.113924
your           0.108962
fine           0.108019
Name: 2, dtype: float64
25 least important terms for role Tax Accountant
portfolio       0.0
portability     0.0
populations     0.0
popular         0.0
poorly          0.0
poor            0.0
polymorphism    0.0
polling         0.0
policy          0.0
policies        0.0
police          0.0
points          0.0
pointless       0.0
pointers        0.0

### Create a list for every rol, containing a dictionary of words and weights for every keyword

In [79]:
word_score_list=[]
i=0
for title in df["questions"]: #Para cada rol
    role_list=[] #Para cada rol creaerme una lista
    for question in title:
        new_dict={} # Para cada pregunta en el rol, crearme un diccionario
        role_list.append(new_dict) # agregar el diccionario de cada pregunta a la lista del rol
        for keyword in df["keywords"][i]: # Para cada keyword en la lista de keywords i
            if keyword in tf_idf_dt_matrix.columns and (keyword in question): #Si e keywords está en el TFIDF matrix y en la pregunta:
                new_dict[keyword]=0 # agregar al diccionario con un valor de 0
    word_score_list.append(role_list)
    i+=1

print(word_score_list[0])
print(len(word_score_list)) 

[{'financial': 0}, {'collected': 0, 'client': 0}, {'client': 0}, {}, {'client': 0, 'clients': 0, 'company': 0}, {'data': 0}, {}, {}, {}, {}, {}, {'client': 0}, {'client': 0}, {}, {}, {'client': 0}]
283


In [80]:
word_score_list=[]
i=0
for title in df["questions"]:
    role_list=[]
    for question in title:
        new_dict={}
        role_list.append(new_dict)
        for keyword in df["keywords"][i]:
            if keyword in tf_idf_dt_matrix.columns and (keyword in question):
                new_dict[keyword]=0
    word_score_list.append(role_list)
    i+=1
        
i=0
for role_list in word_score_list:  
    for quest_dict in role_list:
        for word in quest_dict.keys():
            quest_dict[word]=tf_idf_dt_matrix[word][i]
    i+=1  


print(word_score_list[0])
print(len(word_score_list)) 

[{'financial': 0.05487076842823954}, {'collected': 0.08878830532770851, 'client': 0.19431905367345514}, {'client': 0.19431905367345514}, {}, {'client': 0.19431905367345514, 'clients': 0.04859933011349163, 'company': 0.033684237560223595}, {'data': 0.04410577690110151}, {}, {}, {}, {}, {}, {'client': 0.19431905367345514}, {'client': 0.19431905367345514}, {}, {}, {'client': 0.19431905367345514}]
283


### Create a list for each rol, containing the values of the sum of the keywords weights for each question

In [81]:
list_of_points=[]
for role in word_score_list:
    role_list=[]
    for word_dict in role:
        sum=0
        for word in word_dict.keys():
            sum+=word_dict[word]
        role_list.append(sum)
    
    list_of_points.append(role_list)


print(list_of_points[0])
len(list_of_points)


[0.05487076842823954, 0.2831073590011637, 0.19431905367345514, 0, 0.2766026213471704, 0.04410577690110151, 0, 0, 0, 0, 0, 0.19431905367345514, 0.19431905367345514, 0, 0, 0.19431905367345514]


283

### Create a dictionary for each rol, with the questions and the scores of each one of them

In [83]:
question_score_list=[]
t=0
for title in df["questions"]:
    quest_dict={}
    q=0
    for question in title:
        score=list_of_points[t][q]
        quest_dict[question]=score
        q+=1
    t+=1

    question_score_list.append(quest_dict)


print(question_score_list[0])
print(len(question_score_list))

{"['a debtor has missed six months of payments. when you call the individual, they bring up financial difficulties and get emotional. how do you handle this?": 0.05487076842823954, 'what payback plan would you recommend for a long-term client whose payments are due to be collected in x months?': 0.2831073590011637, 'if you called a client to update them on their payment status and they were aggressive or rude to you, how would you handle it?': 0.19431905367345514, 'can you think of a few common excuses that debtors make? how would you respond to each of them?': 0, 'what collection methods do you know? which technique do you think would be more effective for our company/clients?': 0.2766026213471704, 'how do you keep historical data for each account?': 0.04410577690110151, 'how often do you follow up with delinquent account holders and what’s your approach when you get in touch?': 0, 'mention three things a collection specialist should avoid saying when speaking to a debtor.': 0, 'how d

In [84]:
df["scored_questions"]=question_score_list
df.head()

,title,job description,questions,number_of_questions,keywords,keyword_with_jobdesc,keywords_per_question,scored_questions
0,Collection Specialist,job brief: we are looking for a competent coll...,[['a debtor has missed six months of payments....,16,"[coverage_for, instrument_cluster, comapny, da...","[experience, bi, truck, ability, independent, ...",{'['a debtor has missed six months of payments...,{'['a debtor has missed six months of payments...
1,Billing Analyst,job brief: we are looking for a billing analys...,[['how would you create a report on debts from...,14,"[coverage_for, instrument_cluster, comapny, da...","[experience, bi, organizational, contracts, ab...",{'['how would you create a report on debts fro...,{'['how would you create a report on debts fro...
2,Tax Accountant,job brief: we are looking for a tax accountant...,[['what would you do if you received a b-notic...,14,"[coverage_for, instrument_cluster, comapny, da...","[experience, bi, detail, business, financial, ...",{'['what would you do if you received a b-noti...,{'['what would you do if you received a b-noti...
3,Accounts Receivable Manager,job brief: we are looking for an accounts rece...,[['what would be the first thing you’d check i...,17,"[coverage_for, instrument_cluster, comapny, da...","[experience, bi, advanced, ability, detail, cl...",{'['what would be the first thing you’d check ...,{'['what would be the first thing you’d check ...
4,Cost Accountant,job brief: we are looking for a cost accountan...,"[['if you could use only excel, how would you ...",15,"[coverage_for, instrument_cluster, comapny, da...","[bi, reports, information, detail, finance, re...","{'['if you could use only excel, how would you...","{'['if you could use only excel, how would you..."


In [86]:
first_question=df["scored_questions"][1]

In [87]:
{k: v for k, v in sorted( first_question.items(), key=lambda item: item[1],reverse=True)}

{'how do you keep updated records of accounts receivable from clients?': 0.3364495687160205,
 'how do you ensure financial information remains confidential?': 0.2894625108267521,
 'briefly explain the financial transactions involved in the billing process and your experience with each.': 0.2689573335312531,
 'how would you contact clients who haven’t paid their bills?': 0.24941628701207524,
 'how do you stay up-to-date with new financial regulations? do you read any specific blogs, magazines or newspapers? if so, what are they?': 0.2347102424087359,
 "['how would you create a report on debts from the previous quarter?": 0.19850188637719257,
 'if a long-term client had a debt at the end of the fiscal year, how would you address it with him/her?': 0.11082688408608732,
 'how would you handle an invoice discrepancy with a client?': 0.11082688408608732,
 'how would you make sure we collect invoices in a timely manner?': 0,
 'what accounting software have you used?': 0,
 'what are your respo